# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行


In [ ]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "========================================="


In [ ]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config


In [ ]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT


In [ ]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


# 第六章：逐步思考


## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-3.5-turbo"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程

如果有人叫醒您，然后立即开始问您几个复杂的问题，要求您立即回答，您会表现如何？可能不如给您时间**先思考答案**时表现得好。

猜猜怎么着？GPT也是一样的。

**给GPT时间逐步思考有时会让GPT更准确**，特别是对于复杂任务。

## 🔥 中文提示词逐步思考示例

在中文语境下，逐步思考技术同样非常有效。以下示例展示了如何在中文提示词中应用逐步思考的方法。

### 核心原理
- **让模型慢下来思考**：给大模型时间进行逐步推理
- **结构化思维过程**：通过明确的步骤引导模型思考
- **提高准确性**：特别适用于复杂推理、数学计算、逻辑分析等任务

让我们通过实际示例来学习！


### 示例1：数学问题解决 - 对比基础版与逐步思考版

**背景场景**：解决复杂的数学应用题


In [2]:
# 示例1A：基础版本 - 直接提问
print("📍 基础版本：直接提问数学题")
print("=" * 50)

BASIC_MATH_PROMPT = """
小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？
"""

response_basic = get_completion(BASIC_MATH_PROMPT)
print("GPT回答：")
print(response_basic)
print("\n" + "="*80 + "\n")


📍 基础版本：直接提问数学题
GPT回答：
设小明妈妈一共买了 \( x \) 个苹果。

第一天吃掉了总数的 \( \frac{1}{4} \)，剩下的苹果数为：
\[
x - \frac{1}{4}x = \frac{3}{4}x
\]

第二天吃掉了剩余的 \( \frac{1}{3} \)，剩下的苹果数为：
\[
\frac{3}{4}x - \frac{1}{3} \times \frac{3}{4}x = \frac{3}{4}x \times \frac{2}{3} = \frac{1}{2}x
\]

第三天吃掉了4个苹果，剩下的苹果数为：
\[
\frac{1}{2}x - 4
\]

最后还剩下8个苹果，所以有：
\[
\frac{1}{2}x - 4 = 8
\]

解这个方程：
\[
\frac{1}{2}x = 12
\]
\[
x = 24
\]

所以小明妈妈一共买了24个苹果。




In [3]:
# 示例1B：逐步思考版本 - 引导详细推理过程
print("🧠 逐步思考版本：引导详细推理")
print("=" * 50)

STEP_BY_STEP_MATH_PROMPT = """
小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？

请按照以下步骤一步一步分析：
1. 首先，理解题目条件，设定未知数
2. 然后，分析每天的消费情况
3. 接着，建立数学方程
4. 最后，求解并验证答案

让我一步步来解决这个问题：
"""

response_step_by_step = get_completion(STEP_BY_STEP_MATH_PROMPT)
print("GPT回答：")
print(response_step_by_step)
print("\n" + "="*80 + "\n")


🧠 逐步思考版本：引导详细推理
GPT回答：
好的，我们一步步来解决这个问题。

### 1. 理解题目条件，设定未知数

设小明的妈妈一共买了 \( x \) 个苹果。

### 2. 分析每天的消费情况

- **第一天**：吃掉了总数的 \( \frac{1}{4} \)，即 \( \frac{1}{4}x \) 个苹果。  
  剩下的苹果数为：  
  \[
  x - \frac{1}{4}x = \frac{3}{4}x
  \]

- **第二天**：吃掉了剩余的 \( \frac{1}{3} \)，即 \( \frac{1}{3} \times \frac{3}{4}x = \frac{1}{4}x \) 个苹果。  
  剩下的苹果数为：  
  \[
  \frac{3}{4}x - \frac{1}{4}x = \frac{1}{2}x
  \]

- **第三天**：吃掉了 4 个苹果。  
  剩下的苹果数为：  
  \[
  \frac{1}{2}x - 4
  \]

- **最后**：还剩下 8 个苹果。  
  \[
  \frac{1}{2}x - 4 = 8
  \]

### 3. 建立数学方程

根据最后的条件，我们可以建立方程：  
\[
\frac{1}{2}x - 4 = 8
\]

### 4. 求解并验证答案

解这个方程：  
\[
\frac{1}{2}x - 4 = 8
\]

首先，移项：  
\[
\frac{1}{2}x = 8 + 4
\]
\[
\frac{1}{2}x = 12
\]

两边乘以 2：  
\[
x = 24
\]

所以，小明的妈妈一共买了 24 个苹果。

**验证：**

- **第一天**：吃掉 \( \frac{1}{4} \times 24 = 6 \) 个，剩下 \( 24 - 6 = 18 \) 个。
- **第二天**：吃掉 \( \frac{1}{3} \times 18 = 6 \) 个，剩下 \( 18 - 6 = 12 \) 个。
- **第三天**：吃掉 4 个，剩下 \( 12 - 4 = 8 \) 个。

验证结果与题目条件一致，答案正确。小明的妈妈一共买了 24 个苹果。




### 示例2：情感分析 - 中文版本的逐步思考

**背景场景**：分析中文评论的情感倾向，特别是含有讽刺或双重含义的内容


In [4]:
# 示例2A：基础版本 - 直接分析情感
print("📍 基础版本：直接分析评论情感")
print("=" * 50)

BASIC_SENTIMENT_PROMPT = """
请分析以下评论的情感倾向（正面/负面/中性）：

"这部电影真是太'精彩'了，我看到一半就睡着了，醒来发现还没结束，真是让人'印象深刻'。演员的表演也很'自然'，自然到我都怀疑他们是不是在念台词。总之，这是一部让我'终生难忘'的作品。"
"""

response_basic_sentiment = get_completion(BASIC_SENTIMENT_PROMPT)
print("GPT回答：")
print(response_basic_sentiment)
print("\n" + "="*60 + "\n")


📍 基础版本：直接分析评论情感
GPT回答：
这段评论使用了讽刺的语气，虽然表面上使用了一些积极的词汇（如“精彩”、“印象深刻”、“自然”、“终生难忘”），但结合上下文来看，这些词实际上是在表达负面的情感倾向。评论者提到自己在观看电影时睡着了，醒来后电影还未结束，暗示电影冗长乏味。此外，演员的表演被描述为“自然”到让人怀疑是在念台词，这进一步表明对表演的不满。因此，整体情感倾向是负面的。




In [5]:
# 示例2B：逐步思考版本 - 结构化分析过程
print("🧠 逐步思考版本：结构化情感分析")
print("=" * 50)

STEP_BY_STEP_SENTIMENT_PROMPT = """
请分析以下评论的情感倾向。请先进行逐步分析，然后给出最终判断。

评论：
"这部电影真是太'精彩'了，我看到一半就睡着了，醒来发现还没结束，真是让人'印象深刻'。演员的表演也很'自然'，自然到我都怀疑他们是不是在念台词。总之，这是一部让我'终生难忘'的作品。"

请按照以下步骤分析：

<步骤1>
首先分析表面词汇：识别评论中使用的形容词和描述性词语
</步骤1>

<步骤2>
然后分析语境线索：注意引号、反讽等修辞手法
</步骤2>

<步骤3>
接着分析具体描述：分析用户提到的具体行为和感受
</步骤3>

<步骤4>
最后综合判断：基于以上分析，判断真实的情感倾向
</步骤4>

让我逐步分析这个评论：
"""

response_step_sentiment = get_completion(STEP_BY_STEP_SENTIMENT_PROMPT)
print("GPT回答：")
print(response_step_sentiment)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化情感分析
GPT回答：
<步骤1>
首先分析表面词汇：识别评论中使用的形容词和描述性词语

在评论中，使用了以下形容词和描述性词语：
- "精彩"
- "印象深刻"
- "自然"
- "终生难忘"

这些词汇表面上看起来是积极的，通常用于赞美和肯定。

<步骤2>
然后分析语境线索：注意引号、反讽等修辞手法

评论中使用了引号，例如“精彩”、“印象深刻”、“自然”、“终生难忘”，这些引号通常暗示反讽或讽刺的语气。此外，评论中提到“看到一半就睡着了”、“醒来发现还没结束”，这些语境线索表明评论者对电影的实际体验并不积极。

<步骤3>
接着分析具体描述：分析用户提到的具体行为和感受

评论者提到“看到一半就睡着了”，这表明电影并没有吸引力或足够的精彩来保持观众的注意力。醒来后发现电影还没结束，进一步暗示电影冗长或乏味。对于演员的表演，评论者怀疑他们是否在念台词，这表明表演缺乏真实感或投入。

<步骤4>
最后综合判断：基于以上分析，判断真实的情感倾向

综合以上分析，评论者使用了反讽的手法，通过表面积极的词汇和引号来表达对电影的不满和失望。评论者的真实情感倾向是负面的，对电影的评价是消极的。




### 示例3：商业决策分析 - 中文商业场景

**背景场景**：帮助企业制定市场策略决策


In [6]:
# 示例3A：基础版本 - 直接提问商业建议
print("📍 基础版本：直接询问商业策略")
print("=" * 50)

BASIC_BUSINESS_PROMPT = """
我们公司是一家生产智能手环的科技公司，目前面临激烈的市场竞争。Apple Watch和华为等大品牌占据了主要市场份额，我们的销量正在下滑。请给出市场策略建议。
"""

response_basic_business = get_completion(BASIC_BUSINESS_PROMPT)
print("GPT回答：")
print(response_basic_business)
print("\n" + "="*60 + "\n")


📍 基础版本：直接询问商业策略
GPT回答：
在面对激烈的市场竞争时，尤其是与像Apple和华为这样的强大品牌竞争时，制定有效的市场策略至关重要。以下是一些建议，可以帮助您的公司在智能手环市场中提升竞争力：

1. **差异化产品**：
   - **独特功能**：开发具有独特功能的智能手环，例如专注于特定健康监测（如血糖监测）或提供个性化健身建议。
   - **设计创新**：在设计上进行创新，提供多样化的款式和颜色，以吸引不同的消费者群体。

2. **目标市场细分**：
   - **专注于特定人群**：例如，针对运动爱好者、老年人或儿童设计专门的产品。
   - **区域市场策略**：分析不同地区的市场需求，制定针对性的营销策略。

3. **提升用户体验**：
   - **用户界面优化**：确保手环的操作简单直观，提升用户体验。
   - **应用生态系统**：开发与手环配套的应用程序，提供更多增值服务。

4. **品牌建设与营销**：
   - **品牌故事**：讲述品牌的独特故事，增强品牌的情感吸引力。
   - **社交媒体营销**：利用社交媒体平台进行推广，与用户互动，建立社区。

5. **合作与联盟**：
   - **与健康机构合作**：与医院、健身中心等合作，提升产品的专业性和可信度。
   - **跨界合作**：与时尚品牌或科技公司合作，推出联名产品。

6. **价格策略**：
   - **竞争性定价**：在保证质量的前提下，提供具有竞争力的价格。
   - **促销活动**：定期推出促销活动，吸引消费者购买。

7. **客户反馈与改进**：
   - **收集用户反馈**：通过调查和用户评论收集反馈，持续改进产品。
   - **快速响应市场变化**：根据市场趋势和用户需求快速调整产品和策略。

通过以上策略，您的公司可以在竞争激烈的市场中找到自己的定位，提升品牌知名度和市场份额。




In [7]:
# 示例3B：逐步思考版本 - 结构化商业分析
print("🧠 逐步思考版本：结构化商业策略分析")
print("=" * 50)

STEP_BY_STEP_BUSINESS_PROMPT = """
我们公司是一家生产智能手环的科技公司，目前面临激烈的市场竞争。Apple Watch和华为等大品牌占据了主要市场份额，我们的销量正在下滑。

请按照商业分析框架逐步分析并提出策略建议：

<市场分析>
首先分析当前智能手环市场的竞争格局、用户需求和趋势
</市场分析>

<竞争对手分析>
分析主要竞争对手（Apple Watch、华为等）的优势和劣势
</竞争对手分析>

<自身SWOT分析>
分析我们公司的优势(Strengths)、劣势(Weaknesses)、机会(Opportunities)、威胁(Threats)
</自身SWOT分析>

<差异化策略>
基于分析结果，提出我们的差异化定位和策略
</差异化策略>

<具体执行方案>
给出3-5个具体可执行的市场策略和实施建议
</具体执行方案>

让我按照框架来分析：
"""

response_step_business = get_completion(STEP_BY_STEP_BUSINESS_PROMPT)
print("GPT回答：")
print(response_step_business)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化商业策略分析
GPT回答：
<市场分析>

1. **竞争格局**: 智能手环市场竞争激烈，主要由几大品牌主导，如Apple、华为、三星等。这些品牌拥有强大的品牌影响力和技术研发能力。市场上也有许多中小型企业提供价格更低的产品，试图通过性价比吸引消费者。

2. **用户需求**: 用户对智能手环的需求主要集中在健康监测（如心率、睡眠、运动）、便捷支付、通知提醒等功能。此外，用户越来越关注产品的设计美观、续航能力和与其他设备的兼容性。

3. **趋势**: 市场趋势显示，消费者对健康和健身功能的需求持续增长，尤其是在疫情后。可穿戴设备的智能化和个性化定制也成为趋势。随着技术进步，AI和大数据在智能手环中的应用越来越普遍。

<竞争对手分析>

1. **Apple Watch**:
   - 优势: 强大的品牌影响力、丰富的生态系统、卓越的用户体验和设计、强大的技术支持。
   - 劣势: 价格较高，可能不适合预算有限的消费者。

2. **华为**:
   - 优势: 性价比高、强大的技术研发能力、良好的电池续航、广泛的市场覆盖。
   - 劣势: 在某些市场（如美国）面临政治和贸易限制。

<自身SWOT分析>

1. **优势**:
   - 专注于某些特定功能或用户群体（如专业运动员或老年人）。
   - 具有灵活的产品设计和快速响应市场变化的能力。

2. **劣势**:
   - 品牌知名度较低，市场营销资源有限。
   - 技术研发能力可能不如大品牌。

3. **机会**:
   - 健康和健身市场的持续增长。
   - 新兴市场的扩展机会。

4. **威胁**:
   - 大品牌的市场主导地位。
   - 技术快速变化带来的压力。

<差异化策略>

1. **定位**: 专注于特定用户群体，如专业运动员、老年人或特定健康需求的用户，提供定制化解决方案。

2. **策略**: 强调产品的独特功能和用户体验，利用价格优势和灵活的设计来吸引特定市场。

<具体执行方案>

1. **产品创新**: 开发具有独特功能的智能手环，如针对特定健康问题的监测功能，或与专业运动应用集成的功能。

2. **市场营销**: 加强品牌宣传，通过社交媒体和合作伙伴关系提高品牌知名度。利用用户推荐和口碑营销来扩大影响力。

3. **价格策

### 🎯 逐步思考技巧总结

通过以上中文示例，我们可以看到逐步思考的核心价值：

#### 🔑 关键技巧

1. **使用引导性短语**
   - "让我一步步来解决这个问题："
   - "请按照以下步骤分析："
   - "首先...然后...接着...最后..."

2. **结构化标签**
   - 使用`<步骤1>`、`<分析>`、`<结论>`等XML标签
   - 明确每个步骤的具体任务
   - 保持逻辑递进关系

3. **领域专业框架**
   - 数学：理解→分析→建立方程→求解→验证
   - 商业：市场分析→竞争分析→SWOT→策略→执行
   - 创作：主题→人物→情节→冲突→细节

#### 📈 效果对比

| 方面 | 基础提示 | 逐步思考提示 |
|------|----------|--------------|
| **准确性** | 一般 | 显著提升 |
| **逻辑性** | 跳跃 | 严密连贯 |
| **深度** | 表面 | 深入分析 |
| **可用性** | 需要二次确认 | 直接可用 |

#### 💡 适用场景

- ✅ **复杂推理**：数学计算、逻辑分析
- ✅ **多步骤任务**：商业分析、项目规划
- ✅ **创意工作**：写作、设计、策划
- ✅ **专业分析**：法律、医学、技术评估

---

**💡 记住：给AI时间思考，就像给自己时间思考一样重要！**
